# Access the Rijksmuseum collection via API
### for the Cultural Data Sculpting course at EPFL

In [91]:
import urllib.request, urllib.error, urllib.parse
import csv
from bs4 import BeautifulSoup
import html2text
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json
from urllib.request import urlopen
from PIL import Image

In [477]:
# function to check if value is 'nan'
def isnan(x):
    return (x != x)

In [93]:
# function to open url and return it in squared format 
def getsquared_image(url):
    # open image-url, get width, height
    img = Image.open(urlopen(url))
    width, height = img.size
    # store the bigger one in s
    if width < height:
        s = height
    else:
        s = width
    
    # create empty opaque image file and paste picture
    new_img = Image.new('RGBA', (s,s), (255, 0, 0, 0))
    new_img.paste(img, (round((s-width)/2), round((s-height)/2)))
    
    return new_img

In [94]:
# check the working directory
#os.listdir('../data')

# load artist_qid.csv containing json timeline metadata

In [614]:
# open artist-metadata csv
filename = '../data/artist_qid.csv'
#csv_file = csv.reader(open(filename, "r"))
#df = pd.read_csv(filename)
df.to_csv('../data/artist_qid.csv', index = False, header = True)
df.to_csv('../data/artist_qid_backup2.csv', index = False, header = True)
#df.rename(columns = {'URL-list-small': 'URLs MET'}, inplace = True)
df.columns

Index(['Unnamed: 0', 'Name', 'Wikidata-Q-ID', 'Wikipedia-URL', 'URL-names',
       'Summary', 'Wikipedia-json', 'image', 'thumbnail', 'birth', 'death',
       'timeline_position', 'VIAF', 'wikidata', 'MET Object Indices',
       'REAL MET Object IDS', 'cloudinary thumbnails', 'URLs MET',
       'cloudinary artworks', 'Rijks name', 'URLs Rijk', 'Rijks img dates',
       'Rijks img PIDs', 'Rijks img Inrs'],
      dtype='object')

# load Rijksmuseum Metadatacollection csv

In [461]:
# open Rijksmuseum Metadatacollection csv
filename = '../data/202001-rma-csv-collection.csv'
#csv_file = csv.reader(open(filename, "r"))
df_rijks = pd.read_csv(filename)
#df_rijks

### get some details about dataformat

In [548]:
df_rijks.columns

Index(['objectInventoryNumber', 'objectPersistentIdentifier', 'objectTitle[1]',
       'objectType[1]', 'objectCreator[1]', 'objectCreationDate[1]',
       'objectImage'],
      dtype='object')

## collect artworks from Rijksmuseum (manually)

In [563]:
# check if artist is present in Rijksmuseum image database
creator = df_rijks['objectCreator[1]']
storage = ''
string = ''
for i in range(len(creator)): 
    if creator[i]!=creator[i]:
        continue
    elif ('Cimabue') in creator[i]:                     # search for artist name of choice
        print(i,creator[i])  #, df_rijks.iloc[i]
        store = [i]
        string += str(i) + ','
        #print(i)
liste = string.strip(',').split(',')

In [545]:
# check variable format to put in dictionary
store, string, liste

([659266],
 '613145,613146,620559,636388,659266,',
 ['613145', '613146', '620559', '636388', '659266'])

In [386]:
# cell to help copy part of output manually
string = """595719 Carpi, Girolamo da
"""

# copy result into format
store = []
for i in string.replace('\n', ' ').split(' '):
    if i.isnumeric():
        store.append(i)
    else:
        continue     
#store

In [576]:
# store Rijks_ix in temp_index dict

#temp_index = {'Michelangelo_Buonarroti': [481612, 482348, 490912, 620619, 620663, 623421, 637211] }
#temp_index['Raffaello_da_Montelupo2'] = liste

#temp_index

In [575]:
#backup from time to time
backup3 = temp_index.copy()
len(backup), len(backup2), len(backup3)

(42, 45)

In [590]:
#make new Dataframe for Rijks artists and put artwork rijks csv collection index in column 'ix'

rijks = pd.DataFrame()                            
rijks['URL-names'] = df['URL-names']

ix = [[] for i in range(len(rijks))]              # make empty column 'ix'
rijks['ix'] = ix

for i in (temp_index): # store ix in rijks
    count = 0
    for j in rijks['URL-names']:
        if i == j:
            rijks['ix'][count] = temp_index[i]
            count +=1
        else:
            count +=1   
#rijks[190:260]

In [595]:
# clean the formats in ix list

ix_clean = []
for i in rijks['ix']:
    rix = []
    if i == []:
        rix = i
        ix_clean.append(rix)
    elif len(i) == 1:
        rix = [int(str(i).strip('[').strip(']').strip("'"))]
        ix_clean.append(rix)
        #print(int(str(i).strip('[').strip(']').strip("'")))
    elif len(i) >1:
        rix = [int(j) for j in i]
        ix_clean.append(rix)
        
rijks['ix'] = ix_clean              # overwrite ix with ix_clean
#rijks

In [606]:
#rijks

# fill new rijks dataframe with metadata

In [609]:
rijks_creator = []
rijks_urls = []
rijks_pid = []
rijks_inv_num = []
rijks_art_years = []

count = 0
for j, i in enumerate(rijks['ix']):
    
    urlstore = []
    yearstore = []
    inv_num_store = []
    pid_store = []
    
    if i == []:                 # empty? leave it
        count +=1
        creator = i
    else:                       # else: image urls
        creator = (df_rijks.iloc[i[0]]['objectCreator[1]'])
        
        for j in i:                                    # i = list of artwork - ix
            
            x = (df_rijks.iloc[j]['objectImage'])
            urlstore.append(x)
            
            x = (df_rijks.iloc[j]['objectCreationDate[1]'])
            yearstore.append(x)
            
            x = (df_rijks.iloc[j]['objectInventoryNumber'])
            inv_num_store.append(x)
            
            x = (df_rijks.iloc[j]['objectPersistentIdentifier'])
            pid_store.append(x)
            
        count +=1
    
    rijks_creator.append(creator)
    rijks_urls.append(urlstore)
    rijks_inv_num.append(inv_num_store)
    rijks_pid.append(pid_store)
    rijks_art_years.append(yearstore)

rijks['Rijks img Creator'] = rijks_creator
rijks['Rijks img urls'] = rijks_urls
rijks['Rijks img dates'] = rijks_art_years
rijks['Rijks img Inrs'] = rijks_inv_num
rijks['Rijks img PIDs'] = rijks_pid

#len(rijks_creator), len(rijks_urls), len(rijks_pid), len(rijks_inv_num), len(rijks_art_years)
rijks.columns

Index(['URL-names', 'ix', 'Rijks img Creator', 'Rijks img urls',
       'Rijks img dates', 'Rijks img Inrs', 'Rijks img PIDs'],
      dtype='object')

# append Rijks metadata to df

In [610]:
df.columns

Index(['Unnamed: 0', 'Name', 'Wikidata-Q-ID', 'Wikipedia-URL', 'URL-names',
       'Summary', 'Wikipedia-json', 'image', 'thumbnail', 'birth', 'death',
       'timeline_position', 'VIAF', 'wikidata', 'MET Object Indices',
       'REAL MET Object IDS', 'cloudinary thumbnails', 'URLs MET',
       'cloudinary artworks'],
      dtype='object')

In [622]:
df['Rijks name'] = rijks['Rijks img Creator']
df['URLs Rijk'] = rijks['Rijks img urls']
df['Rijks img dates'] = rijks['Rijks img dates']
df['Rijks img PIDs'] = rijks['Rijks img PIDs']
df['Rijks img Inrs'] = rijks['Rijks img Inrs']
df.columns

Index(['Unnamed: 0', 'Name', 'Wikidata-Q-ID', 'Wikipedia-URL', 'URL-names',
       'Summary', 'Wikipedia-json', 'image', 'thumbnail', 'birth', 'death',
       'timeline_position', 'VIAF', 'wikidata', 'MET Object Indices',
       'REAL MET Object IDS', 'cloudinary thumbnails', 'URLs MET',
       'cloudinary artworks', 'Rijks name', 'URLs Rijk', 'Rijks img dates',
       'Rijks img PIDs', 'Rijks img Inrs'],
      dtype='object')

In [636]:
df['URLs Rijk'][291]

['https://lh6.ggpht.com/4-5Uom4G7n7-HSs_1klLn5AMpUUgMY7zNyHbNABCNc92lXb4aJrd9aLtN7jl7ehdKZOeOT7v9vzPOQbPjnWxnZjqHmo=s0',
 'https://lh6.ggpht.com/-KH6a-xmeRwEcx-YYRbtkfPYiUV6EA5g90ctpFs8kgtRghaZrB6IwXZxKDzQS1xX36mnz_psVGfF4yxrThc4_bkZf-I=s0',
 'https://lh5.ggpht.com/zRIElxfXD182zCF47w11yS05JJb9qAE-bC_UJ3BhYWJ8nlhI-xZAknOeZXLPrdEuhX4miugFp8j2XUILne3uvpxalLA=s0']

In [635]:
help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers:

read_csv(filepath_or_buffer: Union[ForwardRef('PathLike[str]'), str, IO[~T], io.RawIOBase, io.BufferedIOBase, io.TextIOBase, _io.TextIOWrapper, mmap.mmap], sep=<object object at 0x7fc015c254e0>, delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal: str = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_whit

In [634]:
# save

df.to_csv('../data/artist_qid_rijks.csv', index = False, header = True)
test= pd.read_csv('../data/artist_qid_rijks.csv')
type(test['URLs Rijk'][291])

str

# clean till here

In [582]:
#night['Rijks_ix'], rijks['ix']

# Save and open Dataframes as CSV

In [608]:
# open Rijksmuseum Metadatacollection csv
filename = '../data/rijks.csv'
#csv_file = csv.reader(open(filename, "r"))
#df_rijks = pd.read_csv(filename)
rijks.to_csv(filename, index=False, header=True)

In [607]:
rijks[:50]

,URL-names,ix,Rijks img Creator,Rijks img urls,Rijks img dates,Rijks img Inrs,Rijks img PIDs
0,Cimabue,[],[],[],[],[],[]
1,Arnolfo_di_Cambio,[],[],[],[],[],[]
2,Bonanno_Pisano,[],[],[],[],[],[]
3,Nicola_Pisano,[],[],[],[],[],[]
4,Giovanni_Pisano,[],[],[],[],[],[]
5,Andrea_Tafi_(artist),[],[],[],[],[],[]
6,Gaddo_Gaddi,[],[],[],[],[],[]
7,Margaritone_d%27Arezzo,[],[],[],[],[],[]
8,Giotto_di_Bondone,[],[],[],[],[],[]
9,Puccio_Capanna,[],[],[],[],[],[]


In [457]:
x = df_rijks.iloc[611622]['objectImage']
x

'https://lh3.googleusercontent.com/0492XLs21PAQPXtLPcaZ4dNfmRzMXjuJrR7jsCMF_Oux6cpgpAR8I7hzsEDpByzj5bOlXZB8zRLvt_zqjLZhaYyDvw=s0'

# save as json

In [546]:
import json
with open('../data/temp_artist_index2.json', 'w') as output:
    json.dump(temp_index, output)

In [493]:
# search for empty thumbnails
count = 0
storage = []
for i in df['cloudinary thumbnails']:
    if isnan(i):
        if night_cl['Rijks img Creator'][count] == []:
            print(df['URL-names'][count])
            storage.append(count)

        count +=1
    else:
        count +=1
len(storage)

Pagno_di_Lapo_Portigiani
Antonio_Filarete
Mino_del_Reame
Chimenti_Camicia
Baccio_Pontelli
Francesco_di_Pesello
Galasso_Galassi
Ludovico_Mazzolino
Niccolo_Rondinelli
Matteo_Lappoli
Gerino_da_Pistoia
Rocco_Zoppo
Tommaso_Barnabei
Raffaello_da_Montelupo
Giovanni_Antonio_Lappoli
Niccol%C3%B2_Tribolo
Giuliano_Bugiardini
Tiberio_Calcagni
Giovanni_Boscoli


19

In [484]:
df.columns

Index(['Unnamed: 0', 'Name', 'Wikidata-Q-ID', 'Wikipedia-URL', 'URL-names',
       'Summary', 'Wikipedia-json', 'image', 'thumbnail', 'birth', 'death',
       'timeline_position', 'VIAF', 'wikidata', 'MET Object Indices',
       'REAL MET Object IDS', 'cloudinary thumbnails', 'URLs MET',
       'cloudinary artworks'],
      dtype='object')

# final df

In [ ]:
df_f = pd.DataFrame()
df_f['Name'] = df['Name']
df_f['Wikidata-Q-ID'] = df['Wikidata-Q-ID']
df_f['Wiki URL-names'] = df['URL-names']
df_f['Rijks-names'] = df['Rijks img Creator']
df_f[''] = df['']
df_f[''] = df['']
df_f[''] = df['']
df_f[''] = df['']
df_f[''] = df['']
df_f[''] = df['']
df_f[''] = df['']
df_f[''] = df['']
df_f[''] = df['']

In [486]:
df

,Unnamed: 0,Name,Wikidata-Q-ID,Wikipedia-URL,URL-names,Summary,Wikipedia-json,image,thumbnail,birth,death,timeline_position,VIAF,wikidata,MET Object Indices,REAL MET Object IDS,cloudinary thumbnails,URLs MET,cloudinary artworks
0,0,Cimabue,Q15790,https://en.wikipedia.org/wiki/Cimabue,Cimabue,"Cimabue, also known as Cenni di Pepo or Cenni ...","{'type': 'standard', 'title': 'Cimabue', 'disp...",https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,1240,1302,1271,11146779,https://www.wikidata.org/wiki/Q15790,NaN,NaN,https://res.cloudinary.com/dfyzsulq8/image/upl...,['nan'],NaN
1,1,Arnolfo di Lapo,Q315249,https://en.wikipedia.org/wiki/Arnolfo_di_Cambio,Arnolfo_di_Cambio,Arnolfo di Cambio was an Italian architect and...,"{'type': 'standard', 'title': 'Arnolfo di Camb...",https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,1240,1302,1271,265308757,https://www.wikidata.org/wiki/Q315249,NaN,NaN,https://res.cloudinary.com/dfyzsulq8/image/upl...,['nan'],NaN
2,2,Bonanno,Q892084,https://en.wikipedia.org/wiki/Bonanno_Pisano,Bonanno_Pisano,"Bonanno Pisano (Pisa), active in the 1170s and...","{'type': 'standard', 'title': 'Bonanno Pisano'...",https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,9999,9999,9999,95743493,https://www.wikidata.org/wiki/Q892084,NaN,NaN,https://res.cloudinary.com/dfyzsulq8/image/upl...,['nan'],NaN
3,3,Nicola Pisano,Q364413,https://en.wikipedia.org/wiki/Nicola_Pisano,Nicola_Pisano,Nicola Pisano was an Italian sculptor whose wo...,"{'type': 'standard', 'title': 'Nicola Pisano',...",https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,1225,1284,1254,79399625,https://www.wikidata.org/wiki/Q364413,NaN,NaN,https://res.cloudinary.com/dfyzsulq8/image/upl...,['nan'],NaN
4,4,Giovanni Pisano,Q314932,https://en.wikipedia.org/wiki/Giovanni_Pisano,Giovanni_Pisano,"Giovanni Pisano was an Italian sculptor, paint...","{'type': 'standard', 'title': 'Giovanni Pisano...",https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,1248,1316,1282,100174152,https://www.wikidata.org/wiki/Q314932,[313456 313457 315423 315800],"[463414, 463415, 465937, 466326]",https://res.cloudinary.com/dfyzsulq8/image/upl...,['https://images.metmuseum.org/CRDImages/md/we...,['https://res.cloudinary.com/dfyzsulq8/image/u...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,287,Dono Doni,Q714914,https://en.wikipedia.org/wiki/Dono_Doni,Dono_Doni,"Dono Doni, also known as Adone Doni or Dono de...","{'type': 'standard', 'title': 'Dono Doni', 'di...",https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,1500,1575,1538,74702895,https://www.wikidata.org/wiki/Q714914,NaN,NaN,https://res.cloudinary.com/dfyzsulq8/image/upl...,['nan'],NaN
288,288,Cesare Nebbia,Q503575,https://en.wikipedia.org/wiki/Cesare_Nebbia,Cesare_Nebbia,Cesare Nebbia (c.1536–c.1622) was an Italian p...,"{'type': 'standard', 'title': 'Cesare Nebbia',...",https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,1536,1614,1575,13152008,https://www.wikidata.org/wiki/Q503575,[209395 209639 209640 209658 209659 210359 210...,"[340591, 340835, 340836, 340854, 340855, 34157...",https://res.cloudinary.com/dfyzsulq8/image/upl...,['https://images.metmuseum.org/CRDImages/dp/we...,['https://res.cloudinary.com/dfyzsulq8/image/u...
289,289,Niccolò Circignani detto il Pomarancio,Q382762,https://en.wikipedia.org/wiki/Niccol%C3%B2_Cir...,Niccol%C3%B2_Circignani,Niccolò Circignani was an Italian painter of t...,"{'type': 'standard', 'title': 'Niccolò Circign...",https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,1517,1597,1557,95809921,https://www.wikidata.org/wiki/Q382762,[211022],[342245],https://res.cloudinary.com/dfy